In [ ]:
import pandas as pd
import boto3
from io import BytesIO
from datetime import datetime
from openpyxl import load_workbook
from reportlab.pdfgen import canvas
from reportlab.lib.utils import ImageReader



In [3]:
nombre_bucket = "itam-analytics-grb"
s3 = boto3.client("s3")

In [3]:
def calcular_edad(fecha_nac, fecha_ref):
    return fecha_ref.year - fecha_nac.year - ((fecha_ref.month, fecha_ref.day) < (fecha_nac.month, fecha_nac.day))

In [4]:
ruta_carpeta_base_datos = "../data/solicitudes/Base_Datos/"
ruta_archivo_parametros = "../data/solicitudes/Parametros/parametros.xlsx"
ruta_cuotas = "../modelo/experiencia_global.xlsx"
ruta_emisiones = "../data/emisiones.xlsx"
ruta_historico_cotizaciones = "../data/cotizaciones.xlsx"
ruta_carpeta_calculos = "../data/solicitudes/Calculo_Cotizaciones/"

In [5]:
def prima_experiencia_global(ruta_carpeta_base_datos, ruta_archivo_parametros, ruta_cuotas, ruta_carpeta_calculos, 
                             ruta_emisiones, ruta_historico_cotizaciones, nombre_bucket):
    s3 = boto3.client("s3")

    #Se obtiene la ruta de todas las bases de datos de asegurados.
    response = s3.list_objects_v2(Bucket=nombre_bucket, Prefix=ruta_carpeta_base_datos)
    
    if 'Contents' in response:
        lista_archivos_base_datos = [obj['Key'] for obj in response['Contents']]
        
    else:
        print("No se encontraron archivos en esa carpeta.")
    
    #Se carga base de datos que contiene los parametros de las cotizaciones.
    response = s3.get_object(Bucket=nombre_bucket, Key=ruta_archivo_parametros)
    content = response['Body'].read()
    parametros = pd.read_excel(BytesIO(content), engine='openpyxl')
    
    #Se carga base de datos que contiene las cuotas al millar por edad.
    response = s3.get_object(Bucket=nombre_bucket, Key=ruta_cuotas)
    content = response['Body'].read()
    cuotas = pd.read_excel(BytesIO(content), engine='openpyxl')
    
    #Se carga base de datos que contiene la información de las emisiones y su siniestralidad.
    response = s3.get_object(Bucket=nombre_bucket, Key=ruta_emisiones)
    content = response['Body'].read()
    df_emisiones = pd.read_excel(BytesIO(content), engine='openpyxl')

    #Se carga base de datos que contiene la información histórica de las cotizaciones realizadas.
    response = s3.get_object(Bucket=nombre_bucket, Key=ruta_historico_cotizaciones)
    content = response['Body'].read()
    df_hist_cotizaciones = pd.read_excel(BytesIO(content), engine='openpyxl')

    cotizacion = {
        "Contratante": [],
        "Coberturas": [],
        "SumaAsegurada": [],
        "Administracion": [],
        "Agente": [],
        "Comision": [],
        "FormaPago": [],
        "Inicio": [],
        "Fin": [],
        "Renovacion": [],
        "Poliza": [],
        "Prima": [],
        "EdadPromedio": [],
        "SAMI": [],
        "Asegurados": [],
        "Ticket": [],
        "Mes": [],
        "Oficina": [],
        "Evento": []
    }
    
    for j in range(0,len(parametros)):
        
        ruta_base_datos_asegurados = ""
        ruta_guardar_calculo = ""
        rpf = 0
        desc = 0
        prima = 0
        ticket = len(df_hist_cotizaciones) + j + 1
        
        cotizacion["Contratante"].append(parametros["Contratante"][j])
        cotizacion["Coberturas"].append(parametros["Coberturas"][j])
        cotizacion["SumaAsegurada"].append(parametros["SumaAsegurada"][j])
        cotizacion["Administracion"].append(parametros["Administracion"][j])
        cotizacion["Agente"].append(parametros["Agente"][j])
        cotizacion["Comision"].append(parametros["Comision"][j])
        cotizacion["FormaPago"].append(parametros["FormaPago"][j])
        cotizacion["Inicio"].append(parametros["Inicio"][j])
        cotizacion["Fin"].append(parametros["Fin"][j])
        cotizacion["Renovacion"].append(parametros["Renovacion"][j])
        cotizacion["Poliza"].append(parametros["Poliza"][j])
        cotizacion["Ticket"].append(ticket)
        cotizacion["Oficina"].append(parametros["Oficina"][j])

        mes = pd.to_datetime(parametros["Inicio"][j]).month
        #print(mes)

        if mes == 1:
            mes = "Enero"
        elif mes == 2:
            mes = "Febrero"
        elif mes == 3:
            mes = "Marzo"
        elif mes == 4:
            mes = "Abril"
        elif mes == 5:
            mes = "Mayo"
        elif mes == 6:
            mes = "Junio"
        elif mes == 7:
            mes = "Julio"
        elif mes == 8:
            mes = "Agosto"
        elif mes == 9:
            mes = "Septiembre"
        elif mes == 10:
            mes = "Octubre"
        elif mes == 11:
            mes = "Noviembre"
        elif mes == 12:
            mes = "Diciembre"

        cotizacion["Mes"].append(mes)

        ruta_base_datos_asegurados = lista_archivos_base_datos[j]

        #Carga de la base de datos de asegurados
        response = s3.get_object(Bucket=nombre_bucket, Key=ruta_base_datos_asegurados)
        content = response['Body'].read()
        df_calculo = pd.read_excel(BytesIO(content), engine='openpyxl')
        
        fecha_corte = pd.to_datetime(parametros["Inicio"][j])
        df_calculo["Edad"] = df_calculo["Fecha de Nacimiento"].apply(lambda x: calcular_edad(x, fecha_corte))

        if parametros["FormaPago"][j] == "Anual":
            rpf = 0
            num_recibos = 1
        elif parametros["FormaPago"][j] == "Semestral":
            rpf = 0.037
            num_recibos = 2
        elif parametros["FormaPago"][j] == "Trimestral":
            rpf = 0.055
            num_recibos = 4
        elif parametros["FormaPago"][j] == "Mensual":
            rpf = 0.065
            num_recibos = 12

        if parametros["Comision"][j] == .2:
            desc = 0
        elif parametros["Comision"][j] == .19:
            desc = 0.02
        elif parametros["Comision"][j] == .18:
            desc = 0.03
        elif parametros["Comision"][j] == .17:
            desc = 0.04
        elif parametros["Comision"][j] == .16:
            desc = 0.06
        elif parametros["Comision"][j] == .15:
            desc = 0.07
        elif parametros["Comision"][j] == .14:
            desc = 0.09
        elif parametros["Comision"][j] == .13:
            desc = 0.10
        elif parametros["Comision"][j] == .12:
            desc = 0.12
        elif parametros["Comision"][j] == .11:
            desc = 0.13
        elif parametros["Comision"][j] == .10:
            desc = 0.15
        elif parametros["Comision"][j] == .09:
            desc = 0.16
        elif parametros["Comision"][j] == .08:
            desc = 0.18
        elif parametros["Comision"][j] == .07:
            desc = 0.19
        elif parametros["Comision"][j] == .06:
            desc = 0.21
        elif parametros["Comision"][j] == .05:
            desc = 0.22

        if parametros["Coberturas"][j] == "F":
            df_calculo = df_calculo.merge(cuotas[["Edad","Fallecimiento"]], on="Edad", how="left")
            df_calculo["Fallecimiento"] = df_calculo["Fallecimiento"]*(1-desc)*(1+rpf)*parametros["SumaAsegurada"][j]/1000
            prima = df_calculo["Fallecimiento"].sum()
        elif parametros["Coberturas"][j] == "FMA":
            df_calculo = df_calculo.merge(cuotas[["Edad","Fallecimiento","MA"]], on="Edad", how="left")
            df_calculo["Fallecimiento"] = df_calculo["Fallecimiento"]*(1-desc)*(1+rpf)*parametros["SumaAsegurada"][j]/1000
            df_calculo["MA"] = df_calculo["MA"]*(1-desc)*(1+rpf)*parametros["SumaAsegurada"][j]/1000
            prima = df_calculo["Fallecimiento"].sum()+df_calculo["MA"].sum()
        elif parametros["Coberturas"][j] == "FBPAI":
            df_calculo = df_calculo.merge(cuotas[["Edad","Fallecimiento","BPAI"]], on="Edad", how="left")
            df_calculo["Fallecimiento"] = df_calculo["Fallecimiento"]*(1-desc)*(1+rpf)*parametros["SumaAsegurada"][j]/1000
            df_calculo["BPAI"] = df_calculo["BPAI"]*(1-desc)*(1+rpf)*parametros["SumaAsegurada"][j]/1000
            prima = df_calculo["Fallecimiento"].sum()+df_calculo["BPAI"].sum()
        elif parametros["Coberturas"][j] == "FMABPAI":
            df_calculo = df_calculo.merge(cuotas, on="Edad", how="left")
            df_calculo["Fallecimiento"] = df_calculo["Fallecimiento"]*(1-desc)*(1+rpf)*parametros["SumaAsegurada"][j]/1000
            df_calculo["MA"] = df_calculo["MA"]*(1-desc)*(1+rpf)*parametros["SumaAsegurada"][j]/1000
            df_calculo["BPAI"] = df_calculo["BPAI"]*(1-desc)*(1+rpf)*parametros["SumaAsegurada"][j]/1000
            prima = df_calculo["Fallecimiento"].sum()+df_calculo["BPAI"].sum()+df_calculo["MA"].sum()

        if parametros["Renovacion"][j] == "Si":
            siniestralidad = df_emisiones.loc[df_emisiones["Poliza"] == parametros["Poliza"][j], "Siniestralidad"].values[0]
            
            if siniestralidad < 0.50:
                cotizacion["Prima"].append(prima)
                cotizacion["Evento"].append("na")
            else:
                cotizacion["Prima"].append("La siniestralidad está desviada, consulte a un suscriptor")
                cotizacion["Evento"].append("Fuera de política")
                prima = "La siniestralidad está desviada, consulte a un suscriptor"
            
        else:
            cotizacion["Prima"].append(prima)
            cotizacion["Evento"].append("na")            
            
            
        cotizacion["EdadPromedio"].append(df_calculo["Edad"].mean())
        cotizacion["SAMI"].append(parametros["SumaAsegurada"][j])
        cotizacion["Asegurados"].append(df_calculo["Edad"].count())

        #Guardar base de datos con el calculo de la prima
        file_key = ruta_carpeta_calculos + parametros["Contratante"][j] + ".xlsx"
                # Convertir el DataFrame a un archivo Excel en memoria
        buffer = BytesIO()
        df_calculo.to_excel(buffer, index=False, engine='openpyxl')
        buffer.seek(0)  # volver al inicio del buffer
                # Subir el archivo al bucket
        s3.upload_fileobj(buffer, nombre_bucket, file_key)

        #Creación pdf
        bucket_name = nombre_bucket
        imagen_s3_key = "coco/logo_SegurosDelValle.png"
        pdf_s3_key = "coco/solicitudes/formatos_pdf/" + parametros["Contratante"][j] + ".pdf"
        imagen_buffer = BytesIO()
        s3.download_fileobj(bucket_name, imagen_s3_key, imagen_buffer)   #Carga del logo
        imagen_buffer.seek(0)
        pdf_buffer = BytesIO()   # Crear PDF en memoria
        c = canvas.Canvas(pdf_buffer)
        # Campos variables del pdf
        contratante = parametros["Contratante"][j]
        if parametros["Coberturas"][j] == "F":
            coberturas = "FALLECIMIENTO"
        elif parametros["Coberturas"][j] == "FMA":
            coberturas = "FALLECIMIENTO Y MUERTE ACCIDENTAL"
        elif parametros["Coberturas"][j] == "FBPAI":
            coberturas = "FALLECIMIENTO E INVALIDEZ TOTAL"
        elif parametros["Coberturas"][j] == "FMABPAI":
            coberturas = "FALLECIMIENTO, MUERTE ACCIDENTAL E INVALIDEZ TOTAL"
        suma_asegurada = parametros["SumaAsegurada"][j]
        edad_promedio = df_calculo["Edad"].mean()
        administracion = parametros["Administracion"][j]
        SAMI = parametros["SumaAsegurada"][j]
        agente = parametros["Agente"][j]
        vigencia = str(parametros["Inicio"][j]) + "-" + str(parametros["Fin"][j])
        #prima = prima
        forma_pago = parametros["FormaPago"][j]
        asegurados = df_calculo["Edad"].count()
        # Insertar imagen en la parte superior izquierda (ajustar tamaño/posición si es necesario)
        imagen = ImageReader(imagen_buffer)
        c.drawImage(imagen, x=30, y=740, width=200, height=75)
        # Agregar texto
        c.drawString(50, 700, "DESGLOSE DE ESTUDIO DE SEGURO DE VIDA GRUPO")
        c.drawString(50, 685, f"CONTRATANTE: {contratante}")
        c.drawString(50, 670, f"COBERTURAS: {coberturas}")
        c.drawString(50, 655, f"SUMA ASEGURADA: $ {suma_asegurada:,.2f}")
        c.drawString(50, 640, f"EDAD PROMEDIO: {edad_promedio}")
        c.drawString(50, 625, f"SISTEMA DE ADMINISTRACIÓN: {administracion}")
        c.drawString(50, 610, f"SAMI: $ {SAMI:,.2f}")
        c.drawString(50, 595, "CONTRIBUTORIO: NO CONTRIBUTORIO")
        c.drawString(50, 580, "DIVIDENDOS: SIN DIVIDENDOS")
        c.drawString(50, 565, f"AGENTE: {agente}")
        c.drawString(50, 550, f"VIGENCIA: {vigencia}")
        if isinstance(prima, str):
            c.drawString(50, 535, f"PRIMA:  {prima}")
        else:
            c.drawString(50, 535, f"PRIMA: $ {prima:,.2f}")
        c.drawString(50, 520, f"FORMA DE PAGO: {forma_pago}")
        if isinstance(prima, str):
            c.drawString(50, 505, f"PRIMA:  {prima}")
        else:
            c.drawString(50, 505, f"PRIMER RECIBO Y SUBSECUENTES: $ {prima/num_recibos:,.2f}")
        c.drawString(50, 490, f"ASEGURADOS: {asegurados}")
        c.drawString(50, 475, "EDAD DE ACEPTACIÓN PARA LA COBERTURA DE FALLECIMIENTO: HASTA 75 AÑOS")
        c.drawString(50, 460, "EDAD DE ACEPTACIÓN PARA LA COBERTURA DE MUERTE ACCIDENTAL: HASTA 69 AÑOS")
        c.drawString(50, 445, "EDAD DE ACEPTACIÓN PARA LA COBERTURA DE INVALIDEZ TOTAL: HASTA 64 AÑOS")
        c.drawString(50, 415, "ESTA COTIZACIÓN NO REPRESENTA COMPROMISO DE COBERTURA ALGUNA Y TIENE")
        c.drawString(50, 403, "VIGENCIA DE 30 DÍAS A PARTIR DE LA FECHA QUE ES RECIBIDA")
        c.drawString(50, 385, "RECARGO POR PAGO FRACCIONADO (MENSUAL: 6.5%, TRIMESTRAL: 5.5% Y")
        c.drawString(50, 373, "SEMESTRAL: 3.7%)")
        c.drawString(50, 343, "SEGUROS DEL VALLE, S. A., CON DOMICILIO EN AV. RÍO HONDO, NO. 1, COL. ALTAVISTA,")
        c.drawString(50, 331, "CP 08000, ALCALDÍA BENITO JUÁREZ, CDMX PONE A SU DISPOSICIÓN SU AVISO DE")
        c.drawString(50, 319, "PRIVACIDAD INTEGRAL EN LA PÁGINA WEB WWW.SEGUROSDELVALLE.COM.MX Y LE")
        c.drawString(50, 307, "INFORMA QUE SUS DATOS ESTÁN PROTEGIDOS Y SON UTILIZADOS SOLO PARA REGULAR")
        c.drawString(50, 295, "LOS DERECHOS Y OBLIGACIONES QUE SURGEN DE LA CELEBRACIÓN DE SU CONTRATO")
        c.drawString(50, 283, "DE SEGURO.")
        # Finalizar y preparar el buffer
        c.save()
        pdf_buffer.seek(0)
        # Subir PDF generado a S3
        s3.upload_fileobj(pdf_buffer, bucket_name, pdf_s3_key)
        print("✅ PDF subido exitosamente a:", f"s3://{bucket_name}/{pdf_s3_key}")

    complemento_hist_cotizaciones = pd.DataFrame({
        "Ticket": cotizacion["Ticket"],
        "Fecha de Inicio": cotizacion["Inicio"],
        "Mes": cotizacion["Mes"],
        "Oficina": cotizacion["Oficina"],
        "Contratante": cotizacion["Contratante"],
        "Agente": cotizacion["Agente"],
        "Prima": cotizacion["Prima"],
        "Evento": cotizacion["Evento"],
        "Tipo": cotizacion["Renovacion"],
        
    })

    complemento_hist_cotizaciones["Tipo"] = complemento_hist_cotizaciones["Tipo"].replace({
    "Si": "renovación",
    "No": "nuevo"
    })

    complemento_hist_cotizaciones["Prima"] = complemento_hist_cotizaciones["Prima"].replace({
    "La siniestralidad está desviada, consulte a un suscriptor": 0
    })

    #Agregamos el data frame en la parte inferior del archivo cotizaciones
    file_key = ruta_historico_cotizaciones
    response = s3.get_object(Bucket=nombre_bucket, Key=file_key)
    content = response['Body'].read()
    df_hist_cotizaciones = pd.read_excel(BytesIO(content), engine="openpyxl")
    df_completo = pd.concat([df_hist_cotizaciones, complemento_hist_cotizaciones], ignore_index=True)
    buffer = BytesIO()
    df_completo.to_excel(buffer, index=False, engine='openpyxl')
    buffer.seek(0)
    s3.upload_fileobj(buffer, nombre_bucket, file_key)


In [6]:
prima_experiencia_global(ruta_carpeta_base_datos, ruta_archivo_parametros, ruta_cuotas, ruta_carpeta_calculos, ruta_emisiones, ruta_historico_cotizaciones, nombre_bucket)

ClientError: An error occurred (AccessDenied) when calling the ListObjectsV2 operation: Access Denied